<a href="https://colab.research.google.com/github/sw6820/swm_prototype/blob/main/oddiya_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------------------------------------------------------------------
# [셀 1] 초기 설정 및 라이브러리 설치 (가장 먼저 한 번만 실행)
# ------------------------------------------------------------------------------

# 1. 필수 라이브러리 설치
!pip install -q google-generativeai requests beautifulsoup4

# 2. 라이브러리 임포트
import os
import glob
import subprocess
import shutil
import json
import requests
from bs4 import BeautifulSoup
from PIL import Image
import google.generativeai as genai
from google.colab import drive
from google.colab import userdata

# 3. 구글 드라이브 마운트
try:
    drive.mount('/content/drive')
    print("✅ 구글 드라이브 마운트 성공.")
except Exception as e:
    print(f"❗️ 구글 드라이브 마운트 실패: {e}")

# 4. Colab 보안 비밀에서 API 키 가져오기
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

# 5. 경로 및 영상 속성 설정
IMAGE_FOLDER = '/content/drive/MyDrive/images'
MUSIC_FOLDER = '/content/drive/MyDrive/music' # 자동 다운로드된 음악이 저장될 폴더
OUTPUT_VIDEO = 'output_shorts_final.mp4'
TEMP_DIR = "temp_processing"

IMAGE_DURATION = 3
TRANSITION_DURATION = 1
TARGET_FPS = 30

# 6. AI 설정
AVAILABLE_TRANSITIONS = [
    'fade', 'fadeblack', 'fadewhite', 'distance', 'wipeleft', 'wiperight', 'wipeup', 'wipedown',
    'slideleft', 'slideright', 'slideup', 'slidedown', 'circleopen', 'circleclose', 'dissolve',
    'pixelize', 'radial', 'diagtl', 'diagtr', 'diagbl', 'diagbr'
]

# 7. Gemini 모델 초기화
model = None
if GEMINI_API_KEY:
    try:
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel('gemini-2.0-flash')
        print("✅ Gemini API 설정 완료.")
    except Exception as e:
        print(f"❗️ API 설정 중 오류 발생: {e}")
else:
    print("❗️ Colab 보안 비밀(Secrets)에서 'GOOGLE_API_KEY'를 찾을 수 없습니다.")

Mounted at /content/drive
✅ 구글 드라이브 마운트 성공.
✅ Gemini API 설정 완료.


In [ ]:
pwd

'/content'

In [ ]:
ls MUSIC_FOLDER

ls: cannot access 'MUSIC_FOLDER': No such file or directory


In [ ]:
!ls IMAGE_FOLDER

ls: cannot access 'IMAGE_FOLDER': No such file or directory


In [ ]:
# ------------------------------------------------------------------------------
# [셀 2] AI 기능 정의 (업그레이드)
# ------------------------------------------------------------------------------

def get_ai_creative_plan(image_paths):
    """AI가 이미지를 보고 제목, 효과, 음악 스타일을 포함한 영상 계획을 생성합니다."""
    if not model: return None
    print("🤖 AI가 이미지를 분석하여 영상 계획을 생성합니다...")
    try:
        image_parts = [Image.open(p) for p in image_paths]
        num_transitions = len(image_paths) - 1

        prompt = f"""
        You are a creative director. Based on the following images, create a plan for a short video.
        Provide your response strictly in the following JSON format.

        - "title": A short, catchy title for the video.
        - "transitions": A list of {num_transitions} transition effects from {AVAILABLE_TRANSITIONS}.
        - "music_style_description": A detailed description of the perfect music style for this video (e.g., "A calm, emotional piano melody with a slow tempo, suitable for a nostalgic look back at a beautiful trip.").

        {{
          "title": "Your Title Here",
          "transitions": ["effect1", "effect2", ...],
          "music_style_description": "Your detailed music description here"
        }}
        """
        response = model.generate_content([prompt] + image_parts)
        json_str = response.text.strip().split('```json')[1].split('```')[0]
        return json.loads(json_str)
    except Exception as e:
        print(f"❗️ AI 영상 계획 생성 중 오류 발생: {e}")
        return None

def choose_music_with_ai(music_file_list, music_description):
    """AI가 주어진 음악 파일 목록에서 설명과 가장 일치하는 곡을 선택합니다."""
    if not model or not music_file_list or not music_description: return None
    print(f"🎶 AI가 음악 라이브러리에서 가장 어울리는 곡을 선택합니다...")
    try:
        # 파일명에서 확장자와 숫자 등을 제거하여 AI가 더 잘 이해하도록 정리
        cleaned_names = [name.replace('.mp3', '').replace('.wav', '').replace('-', ' ').replace('_', ' ') for name in music_file_list]

        prompt = f"""
        You are a Music Director. Your task is to select the best song for a video.

        1. The desired music style is: "{music_description}"
        2. Here is a list of available music files: {', '.join(cleaned_names)}

        Analyze the file names and choose the ONE file name that best matches the desired music style.
        Your answer must be only the chosen file name from the list, with no extra text.
        """
        response = model.generate_content(prompt)
        chosen_cleaned_name = response.text.strip()

        # AI가 선택한 정리된 이름과 가장 유사한 원본 파일명 찾기
        for original_name in music_file_list:
            if chosen_cleaned_name.replace(' ', '') in original_name.replace('-', '').replace('_', ''):
                print(f"   - AI 선택: {original_name}")
                return os.path.join(MUSIC_FOLDER, original_name)

        # 일치하는 이름을 못 찾으면 첫 번째 곡을 기본값으로 사용
        print("   - AI가 선택한 곡을 찾지 못해 첫 번째 곡을 기본으로 사용합니다.")
        return os.path.join(MUSIC_FOLDER, music_file_list[0])

    except Exception as e:
        print(f"❗️ AI 음악 선택 중 오류 발생: {e}")
        # 오류 발생 시 첫 번째 곡을 기본값으로 사용
        return os.path.join(MUSIC_FOLDER, music_file_list[0])

In [ ]:
# ------------------------------------------------------------------------------
# [셀 3] FFmpeg 동영상 처리 기능 정의 (함수 선언)
# ------------------------------------------------------------------------------

def run_ffmpeg(command):
    """FFmpeg 명령어를 실행하고 오류를 처리하는 헬퍼 함수입니다."""
    try:
        subprocess.run(command, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        print("\n❗️ FFmpeg 실행 중 오류가 발생했습니다.")
        print("--- FFmpeg Command ---")
        print(' '.join(f"'{arg}'" if ' ' in arg else arg for arg in command))
        print("\n--- FFmpeg Error ---")
        print(e.stderr)
        raise e

def create_silent_video(image_paths, transitions):
    """순차적 방식으로 전환 효과가 적용된 무음 영상을 생성합니다."""
    print("🎬 무음 영상 제작을 시작합니다...")
    base_video_path = os.path.join(TEMP_DIR, "step_0.mp4")

    # 1. 첫 이미지로 기반 영상 생성
    command = [
        'ffmpeg', '-y', '-loop', '1', '-t', str(IMAGE_DURATION), '-i', image_paths[0],
        '-vf', f"scale=1920:1080:force_original_aspect_ratio=decrease,pad=1920:1080:(ow-iw)/2:(oh-ih)/2,setsar=1,format=yuv420p,fps={TARGET_FPS}",
        '-c:v', 'libx264', '-r', str(TARGET_FPS), base_video_path
    ]
    run_ffmpeg(command)

    # 2. 나머지 이미지를 순차적으로 합성
    for i in range(1, len(image_paths)):
        input_video = base_video_path
        output_video_temp = os.path.join(TEMP_DIR, f"step_{i}.mp4")
        video_duration = (i * IMAGE_DURATION) - ((i - 1) * TRANSITION_DURATION)

        filter_complex_str = (
            f"[0:v]settb=AVTB[v0];"
            f"[1:v]scale=1920:1080:force_original_aspect_ratio=decrease,pad=1920:1080:(ow-iw)/2:(oh-ih)/2,setsar=1,format=yuv420p,fps={TARGET_FPS},settb=AVTB[v1];"
            f"[v0][v1]xfade=transition={transitions[i-1]}:duration={TRANSITION_DURATION}:offset={video_duration - TRANSITION_DURATION}"
        )
        command = [
            'ffmpeg', '-y', '-i', input_video, '-loop', '1', '-t', str(IMAGE_DURATION), '-i', image_paths[i],
            '-filter_complex', filter_complex_str,
            '-c:v', 'libx264', '-r', str(TARGET_FPS), output_video_temp
        ]
        run_ffmpeg(command)
        base_video_path = output_video_temp
    return base_video_path

def add_music_to_video(silent_video_path, music_path, final_output_path):
    """무음 영상에 배경 음악을 합성합니다."""
    if not music_path or not os.path.exists(music_path):
        print("⚠️ 음악 파일이 없어 음악 없이 영상을 최종 저장합니다.")
        shutil.copy(silent_video_path, final_output_path)
        return

    print(f"🎵 배경 음악을 영상에 추가합니다...")
    command = [
        'ffmpeg', '-y', '-i', silent_video_path, '-i', music_path,
        '-c:v', 'copy',
        '-c:a', 'aac',
        '-shortest',
        final_output_path
    ]
    run_ffmpeg(command)

In [ ]:
# ------------------------------------------------------------------------------
# [셀 4] 메인 파이프라인 실행 (이 셀을 실행하여 전체 프로세스 시작)
# ------------------------------------------------------------------------------

if __name__ == "__main__":
    if os.path.exists(TEMP_DIR): shutil.rmtree(TEMP_DIR)
    os.makedirs(TEMP_DIR)

    image_files = sorted(glob.glob(os.path.join(IMAGE_FOLDER, '*.jpeg')))
    music_files = os.listdir(MUSIC_FOLDER)

    if len(image_files) < 2:
        print(f"❗️'{IMAGE_FOLDER}' 폴더에 최소 2개 이상의 .jpeg 파일이 필요합니다.")
    elif not music_files:
        print(f"❗️'{MUSIC_FOLDER}' 폴더에 음악 파일이 없습니다.")
    else:
        try:
            # 1. AI 영상 계획 수립
            plan = get_ai_creative_plan(image_files)
            if not plan:
                raise Exception("AI로부터 영상 계획을 받아오지 못했습니다.")

            video_title = plan.get('title', "AI Title Failed")
            transitions = plan.get('transitions', ['fade'] * (len(image_files) - 1))
            music_description = plan.get('music_style_description', "")

            print("\n" + "="*50)
            print("📊 AI 영상 제작 계획:")
            print(f"  - 제목: {video_title}")
            print(f"  - 전환 효과: {', '.join(transitions)}")
            print(f"  - 추천 음악 스타일: '{music_description}'")
            print("="*50 + "\n")

            # 2. AI가 로컬 음악 라이브러리에서 최적의 곡 선택
            chosen_music_path = choose_music_with_ai(music_files, music_description)

            # 3. 무음 영상 제작
            final_silent_video = create_silent_video(image_files, transitions)

            # 4. 음악 추가하여 최종 영상 완성
            add_music_to_video(final_silent_video, chosen_music_path, OUTPUT_VIDEO)

            # 5. 최종 결과 출력
            print("\n🎉 최종 영상 생성 완료! 🎉")
            print(f"💾 저장된 파일: {OUTPUT_VIDEO}")
            print("왼쪽 파일 탐색기에서 새로고침하여 확인 후 다운로드할 수 있습니다.")

        except Exception as e:
            print(f"\n💥 최종 프로세스 중단: {e}")
        finally:
            print("🧹 임시 파일을 정리합니다.")
            if os.path.exists(TEMP_DIR):
                shutil.rmtree(TEMP_DIR)

🤖 AI가 이미지를 분석하여 영상 계획을 생성합니다...

📊 AI 영상 제작 계획:
  - 제목: Gangneung Moments
  - 전환 효과: fade, slideup, slideright, dissolve
  - 추천 음악 스타일: 'An upbeat, breezy indie pop track with acoustic elements and a positive, summery vibe. The melody should be light and cheerful, perfect for capturing the feeling of a fun beach vacation. Include some electronic elements, like synth pads or light percussion, to add a modern touch.'

🎶 AI가 음악 라이브러리에서 가장 어울리는 곡을 선택합니다...
   - AI 선택: summer-bliss-vacation-travel-vlog-music-367117.mp3
🎬 무음 영상 제작을 시작합니다...
🎵 배경 음악을 영상에 추가합니다...

🎉 최종 영상 생성 완료! 🎉
💾 저장된 파일: output_shorts_final.mp4
왼쪽 파일 탐색기에서 새로고침하여 확인 후 다운로드할 수 있습니다.
🧹 임시 파일을 정리합니다.
